<a href="https://colab.research.google.com/github/Aparnamol-KS/Text2LaTex/blob/main/Text2LaTex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# Extract relevant columns
data = df[["spoken_English", "equation"]]
print(data)

                                spoken_English           equation
0         Belongs to the interval from 3 to 7.  $ \in [ 3 , 7 ] $
1                                       d eta.          $ d\eta $
2                              a function of t        $ a ( t ) $
3                         sigma squared sub t.     $ \sigma^2_t $
4                                     delta t.        $ \deltat $
...                                        ...                ...
23195826                                     A              $ A $
23195827                                     A              $ A $
23195828                                     B              $ B $
23195829                                     A              $ A $
23195830                                     A              $ A $

[23195831 rows x 2 columns]


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("aaai25withanonymous/MathBridge")

# # Save the dataset to a local file
# dataset.save_to_disk("mathbridge_dataset")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.parquet:   0%|          | 0.00/968M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/23195831 [00:00<?, ? examples/s]

In [ ]:

import pandas as pd


# Convert to Pandas DataFrame
df = dataset["train"].to_pandas()

# Save to CSV
df.to_csv("mathbridge_dataset.csv", index=False)

print("Dataset saved as CSV!")

Dataset saved as CSV!


In [ ]:
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# Copy the CSV file to Google Drive
!cp mathbridge_dataset.csv /content/drive/MyDrive/

print("Dataset uploaded to Google Drive!")

Dataset uploaded to Google Drive!


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

# Load the dataset in streaming mode
dataset = load_dataset("csv", data_files={"train": "mathbridge_dataset.csv"}, streaming=True)

In [ ]:
import os
import csv

def split_and_save_streaming_dataset(dataset, train_ratio=0.9, output_dir="split_dataset"):
    os.makedirs(output_dir, exist_ok=True)

    with open(os.path.join(output_dir, "train.csv"), "w", newline="", encoding="utf-8") as train_file, \
         open(os.path.join(output_dir, "validation.csv"), "w", newline="", encoding="utf-8") as val_file:

        train_writer = csv.writer(train_file)
        val_writer = csv.writer(val_file)

        # Write headers
        headers = ["spoken_English", "equation"]
        train_writer.writerow(headers)
        val_writer.writerow(headers)

        # Process each row
        for i, example in enumerate(dataset["train"]):
            spoken_english = example["spoken_English"]
            equation = example["equation"]

            # Ensure proper escaping of commas and special characters
            if i % int(1 / (1 - train_ratio)) == 0:
                val_writer.writerow([spoken_english, equation])
            else:
                train_writer.writerow([spoken_english, equation])

# Run the dataset splitting function again
split_and_save_streaming_dataset(dataset, train_ratio=0.9, output_dir="split_dataset")


In [ ]:
from datasets import load_dataset
split_dataset = load_dataset("csv", data_files={
    "train": "split_dataset/train.csv",
    "validation": "split_dataset/validation.csv"
}, streaming=True, delimiter=",")


In [ ]:
import pandas as pd

# Clean train dataset
df_train = pd.read_csv("split_dataset/train.csv", on_bad_lines="skip")  # Removes problematic rows
df_train.to_csv("split_dataset/train_cleaned.csv", index=False)

# Clean validation dataset
df_val = pd.read_csv("split_dataset/validation.csv", on_bad_lines="skip")  # Removes problematic rows
df_val.to_csv("split_dataset/validation_cleaned.csv", index=False)


In [ ]:
from datasets import load_dataset
split_dataset = load_dataset("csv", data_files={
    "train": "split_dataset/train_cleaned.csv",
    "validation": "split_dataset/validation_cleaned.csv"
}, streaming=True, delimiter=",")


In [ ]:
from transformers import T5Tokenizer

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Function to preprocess the dataset again
def preprocess_function(examples):
    inputs = [f"Translate to LaTeX: {text}" for text in examples["spoken_English"]]
    targets = examples["equation"]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply new tokenization to match the new settings
tokenized_dataset = split_dataset.map(preprocess_function, batched=True, batch_size=8)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Optimized code ( Run this )

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load model
model_name = "t5-small"

# Set batch size (adjust based on Colab memory)
batch_size = 500000  # Process 500K rows at a time
total_train_samples = 20876282  # Full training dataset size

# Train in chunks
for i in range(0, total_train_samples, batch_size):
    print(f"Training on batch {i} to {min(i + batch_size, total_train_samples)}...")

    # Load model (resume training if not the first batch)
    model = T5ForConditionalGeneration.from_pretrained("./fine-tuned-t5") if i > 0 else T5ForConditionalGeneration.from_pretrained(model_name)

    # Training arguments (optimized for Colab)
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="no",
        learning_rate=2e-5,
        per_device_train_batch_size=1,  # Small batch size to prevent crashes
        gradient_accumulation_steps=8,  # Accumulate gradients to improve training
        num_train_epochs=1,  # One epoch per batch
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=500,
        save_steps=10000,  # Save model every 10K steps
        save_total_limit=1,
        max_steps=200,  # Limit steps per batch
        fp16=True,  # Enable mixed precision training
        dataloader_num_workers=1,
        report_to="none",
    )

    # **Fix: Use `.skip()` and `.take()` for IterableDataset**
    small_train_dataset = tokenized_dataset["train"].skip(i).take(batch_size)

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
    )

    # Train on the batch
    trainer.train()

    # Save model after each batch
    trainer.save_model("./fine-tuned-t5")
    tokenizer.save_pretrained("./fine-tuned-t5")


Training on batch 0 to 500000...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


Training on batch 500000 to 1000000...


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil
shutil.copytree("./results", "/content/drive/MyDrive/t5_checkpoints", dirs_exist_ok=True)
print("Checkpoint copied to Google Drive!")


Checkpoint copied to Google Drive!


In [ ]:
import shutil
shutil.copytree("./split_dataset", "/content/drive/MyDrive/t5_data", dirs_exist_ok=True)
print("Checkpoint copied to Google Drive!")

Checkpoint copied to Google Drive!


In [ ]:
import shutil

shutil.copy("mathbridge_dataset.csv", "/content/drive/MyDrive/t5_data/mathbridge_dataset.csv")
print("Dataset copied to Google Drive!")


Dataset copied to Google Drive!


# Load the model for next batch

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/t5_checkpoints/checkpoint-200")


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = "/content/drive/MyDrive/t5_checkpoints/fine-tuned-t5"

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Load model
model = T5ForConditionalGeneration.from_pretrained(model_path)


In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad and param.data.sum() == 0:
        print(f"Layer {name} has only zero values!")


In [ ]:
output_ids = model.generate(input_ids, max_length=50, do_sample=True, temperature=0.7)


In [ ]:
input_text = "Convert to LaTeX: integral of x squared dx"
tokens = tokenizer(input_text, return_tensors="pt")
print(tokens)


{'input_ids': tensor([[1193, 3027,   12,  325,  382,   15,    4,   10, 9074,   13,    3,  226,
         2812,   26,    3,   26,  226,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = "/content/drive/MyDrive/t5_checkpoints/fine-tuned-t5"  # Try fine-tuned version
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

input_text = "Convert to LaTeX: integral of x squared dx"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

output_ids = model.generate(input_ids, max_length=50)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated Output:", output_text)


Generated Output: 
